<a href="https://colab.research.google.com/github/frank14/Reconocimiento_Facial_Y_Detector_De_Mentiras/blob/main/Reconocimiento_Facial_Y_Detector_De_Mentiras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sistema detector de mentiras**

El presente proyecto utiliza OpenCV y Haar Cascade para la detección y clasificación de rostros.

Para el reconocimiento facial emplearemos: Eigenfaces y el Local Binary Patterns Histograms.

Y finalmente para la detección de mentiras se usan los modelos pre-entredados para detectar las pupilas y los ojos para determinar por medio de un simple cálculo, donde dependiendo la posición de los ojos en cuanto a la posición inicial menos la final la posibilidad de estar mintiendo.

# **Acceso a la cámara web para obtener imágenes y vídeo**

Este cuaderno repasará cómo acceder y ejecutar código en imágenes y vídeos tomados con tu webcam.

Para el propósito de este proyecto vamos a utilizar **Haar Cascade** de **OpenCV** para hacer la detección de rostros capturados a través de la cámara de nuestra computadora.

In [ ]:
# Importar dependencias
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import os
import numpy as np
import PIL
import io
import html
import time
from time import time
import datetime
from tqdm.auto import tqdm

# **Funciones de ayuda**

A continuación se presentan algunas funciones de ayuda para realizar la conversión entre diferentes tipos y formatos de datos de imagen.

In [ ]:
# Función para convertir el objeto JavaScript en una imagen OpenCV
def js_to_image(js_reply):
  """
  Params:
          js_reply: Objeto JavaScript que contiene la imagen de la cámara web
  Returns:
          img: OpenCV BGR imagen
  """
  # Decodificar imagen base64
  image_bytes = b64decode(js_reply.split(',')[1])
  # Convertir bytes a array numpy
  jpg_as_np = np.frombuffer(image_bytes, dtype = np.uint8)
  # Decodificar la matriz numpy en la imagen BGR de OpenCV
  img = cv2.imdecode(jpg_as_np, flags = 1)

  return img

# Función para convertir la imagen del cuadro delimitador del rectángulo de OpenCV en una cadena de bytes base64 para superponerla a la secuencia de vídeo
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Matriz Numpy (píxeles) que contiene el rectángulo a superponer en el flujo de vídeo.
  Returns:
        bytes: Cadena de bytes de imagen Base64
  """
  # Convertir la matriz en imagen PIL
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # Formatear bbox para retornar en formato png
  bbox_PIL.save(iobuf, format = 'png')
  # Format resultado resgresado como una cadena
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

# Función para convertir la imagen del cuadro del ojo izquierdo delimitador del rectángulo de OpenCV en una cadena de bytes base64 para superponerla a la secuencia de vídeo
def bbox_eye_left_to_bytes(bbox_array_eye_left):
  """
  Params:
          bbox_array: Matriz Numpy (píxeles) que contiene el rectángulo a superponer en el flujo de vídeo.
  Returns:
        bytes: Cadena de bytes de imagen Base64
  """
  # Convertir la matriz en imagen PIL
  bbox_eye_left_PIL = PIL.Image.fromarray(bbox_array_eye_left, 'RGBA')
  iobuf = io.BytesIO()
  # Formatear bbox para retornar en formato png
  bbox_eye_left_PIL.save(iobuf, format = 'png')
  # Format resultado resgresado como una cadena
  bbox_bytes_eye_left = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes_eye_left

# Función para convertir la imagen del cuadro del ojo derecho delimitador del rectángulo de OpenCV en una cadena de bytes base64 para superponerla a la secuencia de vídeo
def bbox_eye_right_to_bytes(bbox_array_eye_right):
  """
  Params:
          bbox_array: Matriz Numpy (píxeles) que contiene el rectángulo a superponer en el flujo de vídeo.
  Returns:
        bytes: Cadena de bytes de imagen Base64
  """
  # Convertir la matriz en imagen PIL
  bbox_eye_right_PIL = PIL.Image.fromarray(bbox_array_eye_right, 'RGBA')
  iobuf = io.BytesIO()
  # Formatear bbox para retornar en formato png
  bbox_eye_right_PIL.save(iobuf, format = 'png')
  # Format resultado resgresado como una cadena
  bbox_bytes_eye_right = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes_eye_right

# **Haar Cascade Classifier**

Para este proyecto vamos a ejecutar un algoritmo simple de detección de objetos llamado **Haar Cascade** en nuestras imágenes y vídeo obtenido de nuestra cámara web. **OpenCV** tiene un modelo de detección de rostros pre-entrenado en **Haar Cascade**.

In [ ]:
# Inicializar el modelo de detección de rostros y ojos en Haar Cascade
# Entre mejor sean los modelos, mejor serán los resultados
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml'))
eye_left_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml'))
eye_right_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_righteye_2splits.xml'))

# **Imágenes de la cámara web**

Ejecutar código en imágenes tomadas de la cámara web es bastante sencillo. Utilizaremos código dentro de **Google Colab's Code Snippets** que tiene una variedad de funciones de código útiles para realizar varias tareas.

Utilizaremos el fragmento de código de **Camera Capture** para utilizar la cámara web de tu computadora.

In [ ]:
def take_photo(filename = 'photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Tomar Foto';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Redimensiona la salida para que se ajuste al elemento de vídeo.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Espere a que se haga clic en Capturar.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # Obtener datos fotográficos
  data = eval_js('takePhoto({})'.format(quality))
  # Obtener una imagen en formato OpenCV
  img = js_to_image(data) 
  # Imagen en escala de grises
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # Obtener las coordenadas de la caja delimitadora de la cara mediante la cascada de Haar
  faces = face_cascade.detectMultiScale(gray)
  # Dibujar el cuadro delimitador de la cara en la imagen
  for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(128, 255, 0),2)
  # Guardar la imagen
  cv2.imwrite(filename, img)

  return filename

## **Ejecutar la captura con la camara**

In [ ]:
try:
  # Nombre del archivo a guardar
  filename = take_photo('photo.jpg')
  # Mostrando por consola el nombre del archivo capturado
  print('Saved to {}'.format(filename))
  
  # Muestra la imagen que se acaba de tomar
  display(Image(filename))
except Exception as err:
  # Se lanzarán errores si el usuario no tiene una cámara web 
  # O si no concede el permiso acceso al uso de la misma
  print('Se presenta el siguiente error: ' + str(err))

# **Vídeo de cámara web**

Ejecutar el código en el vídeo de la cámara web es un poco más complejo que las imágenes. Tenemos que iniciar un flujo de vídeo utilizando nuestra cámara web como entrada. A continuación, ejecutamos cada fotograma a través de nuestro programa **(detección de rostros)** y creamos una imagen superpuesta que contiene el cuadro delimitador de la detección. A continuación, superponemos la imagen del cuadro delimitador en el siguiente fotograma de nuestro flujo de vídeo.

In [ ]:
# JavaScript para crear correctamente nuestro flujo de vídeo en directo utilizando nuestra cámara web como entrada
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Estado:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No existen datos';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: green; font-weight: bold;">' +
          'Cuando haya terminado, haga clic aquí o en el vídeo para detener esta demostración</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

## **Iniciar la transmisión de vídeo desde la cámara web**

In [ ]:
# Iniciar la transmisión de vídeo desde la cámara web
video_stream()
# Etiqueta para el vídeo
label_html = 'Capturando...'
# Inicializar el cuadro delimitador para que esté vacío
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convertir la respuesta JS en imagen OpenCV
    img = js_to_image(js_reply["img"])

    # Crear una superposición transparente para el cuadro delimitador
    bbox_array = np.zeros([480,640,4], dtype = np.uint8)

    # Imagen en escala de grises para la detección de rostros
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Obtener las coordenadas de la región de la cara
    faces = face_cascade.detectMultiScale(gray)
    # Obtener el cuadro delimitador de la cara para la superposición
    for (x,y,w,h) in faces:
      bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(128,255,0),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # Convertir la superposición de bbox en bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # Actualizar bbox para que el siguiente cuadro tenga una nueva superposición
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

# **Reconocimiento Facial**

Para poder llevar a cabo el reconocimiento facial, en primer lugar necesitaremos recolectar los datos, es decir los rostros de las personas que se desee reconocer, posteriormente entrenaremos el clasificador, para finalmente probarlo. Para todo este proceso será necesario usar la  detección de rostros con **haarcascades** que habíamos visto anteriormente.

## **Almacenar datos de personas manualmente**

In [ ]:
# Se solicita el nombre de la persona a la que se realizará el reconocimiento
personName = input('Nombre de la persona a reconocer: ')
# Se crea la carpeta donde se almacenaran las imagenes
dataPath = 'Reconocimiento Facial/Data'
personPath = dataPath + '/' + personName
# Si no existe la carpeta se crea
if not os.path.exists(personPath):
  print('Carpeta creada: ',personPath)
  os.makedirs(personPath)

def take_photo(filename = dataPath + '/' + personName, quality=0.8):
  js = Javascript('''
    function delay(n){
    return new Promise(function(resolve){
        setTimeout(resolve,n*1000);
    });
    } 
    async function takePhoto(quality) {
      const div = document.createElement('div');

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Redimensiona la salida para que se ajuste al elemento de vídeo.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Espera de 5 segundos para que el obturador capture correctamente la imagen.
      await delay(5);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)

  # Obtener datos fotográficos
  data = eval_js('takePhoto({})'.format(quality))
  # Obtener una imagen en formato OpenCV
  img = js_to_image(data) 
  # Imagen en escala de grises
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  print(gray.shape)
  # Obtener las coordenadas de la caja delimitadora de la cara mediante la cascada de Haar
  faces = face_cascade.detectMultiScale(gray)
  # Dibujar el cuadro delimitador de la cara en la imagen
  for (x,y,w,h) in faces:
    img = cv2.rectangle(img,(x,y),(x+w,y+h),(128, 255, 0),2)
  # Guardar la imagen
  cv2.imwrite(filename, img)

  return filename

Nombre de la persona a reconocer: Francisco
Carpeta creada:  Reconocimiento Facial/Data/Francisco


In [ ]:
# Habilitar capturas por video
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
# Inicializando el contador para colocar diferentes nombres en las imágenes
count = 0
# Se conculta al usuario la cantidad de fotos a tomar se recomiendan pocas para pruebas rápidas
photo = int(input('Ingrese la cantidad de imagenes a capturar: '))
# Se ejecutara el ciclo según la cantidad indicada por el usuario

while True:
  # Leyendo las capturas
  ret, frame = cap.read()
  try:
    # Nombre y ruta para almacenar las imágenes de entrenamiento
    filename = take_photo(personPath + '/' + 'rostro_{}.jpg'.format(count + 1))
    # Incremetando en 1 el contador por cada imágen capturada
    count = count + 1
    # Mostrando por consola el nombre del archivo capturado
    print('Saved to {}'.format(filename))
    # Tiempo de espera de 1 ms entre foto y foto
    k =  cv2.waitKey(1)
    # Condición para finalizar el ciclo
    if k == 27 or count >= photo:
        break
    # Muestra la imagen que se acaba de tomar
    display(Image(filename))
  except Exception as err:
    # Se lanzarán errores si el usuario no tiene una cámara web 
    # O si no concede el permiso acceso al uso de la misma
    print('Se presenta el siguiente error: ' + str(err))

## **Descargar datos de reconocimiento**

In [ ]:
import os
import requests
from multiprocessing.pool import ThreadPool

def url_response(url):
    path, url = url
    r = requests.get(url, stream = True)
    with open(path, 'wb') as f:
        for ch in r:
            f.write(ch)

# Se solicita el nombre de la persona a la que se realizará el reconocimiento
personName = input('Nombre de la persona a reconocer: ')
# Se crea la carpeta donde se almacenaran las imagenes
dataPath = 'Reconocimiento Facial/Data'
personPath = dataPath + '/' + personName + '/'
# Si no existe la carpeta se crea
if not os.path.exists(personPath):
  print('Carpeta creada: ',personPath)
  os.makedirs(personPath)

urls = [(personPath + "Event1", "https://www.python.org/events/python-events/805/"),
(personPath + "Event2", "https://www.python.org/events/python-events/801/"),
(personPath + "Event3", "https://www.python.org/events/python-events/790/"),
(personPath + "Event4", "https://www.python.org/events/python-events/798/"),
(personPath + "Event5", "https://www.python.org/events/python-events/807/"),
(personPath + "Event6", "https://www.python.org/events/python-events/807/"),
(personPath + "Event7", "https://www.python.org/events/python-events/757/"),
(personPath + "Event8", "https://www.python.org/events/python-user-group/816/")]

ThreadPool(9).imap_unordered(url_response, urls)

print("Descargando imagenes...")
for i in tqdm(range(1000)):
  print("", end = '\r')

Nombre de la persona a reconocer: Lorena
Carpeta creada:  Reconocimiento Facial/Data/Lorena/
Descargando imagenes...


## **Preparando Los Datos Para Entrenar**

Antes de proceder con el entrenamiento es necesario tener cada una de las imágenes con una etiqueta asociada a la persona a la que pertenecen los rostros. Por ello, por ejemplo cuando leamos la carpeta **‘Persona 1’** todas esas imágenes se les asignará etiqueta 0, luego a todas imágenes de los rostros de **‘Persona 2’** se asignará 1,  de **‘Persona 3’** se asignará 2, y así sucesivamente. Con cada una de estas etiquetas le haremos saber al computador que las imágenes le corresponden a personas diferentes.

In [ ]:
# Se indica la carpeta donde se almacenaron las imagenes de entrenamiento
dataPath = 'Reconocimiento Facial/Data'
# Se listan las personas por carpetas
peopleList = os.listdir(dataPath)
# Se imprime el listado de personas en el sistema
print('Lista de personas: ', peopleList)
# Declaramos labels en este se almacenarán las etiquetas correspondientes a cada imagen según la persona
labels = []
# Declaramos facesData en donde se almacenará cada una de las imágenes de los rostros
facesData = []
# Estableceremos un contador label en 0, para conforme se termine de leer las imágenes de una persona, cambie a otro valor. Esto ayudará al clasificador a entender que se tarta de diferentes personas
label = 0
for nameDir in peopleList:
    personPath = dataPath + '/' + nameDir
    print('Leyendo las imágenes')
    for fileName in os.listdir(personPath):
        print('Rostros: ', nameDir + '/' + fileName)
        labels.append(label)
        facesData.append(cv2.imread(personPath+'/'+fileName,0))
    label = label + 1

Lista de personas:  ['RaulEvent1', '.ipynb_checkpoints', 'RaulEvent6', 'RaulEvent7', 'Raul', 'RaulEvent3', 'RaulEvent4', 'RaulEvent8', 'RaulEvent2', 'Francisco', 'RaulEvent5', 'Lorena']
Leyendo las imágenes


NotADirectoryError: ignored

## **Métodos para entrenar el reconocedor**

In [ ]:
#@title **Seleccione una de las siguientes opciones:** { display-mode: "form", run: "auto"}
opciones = "EigenFaces" #@param ["EigenFaces", "LBPH"]

print("La opción seleccionada fue: " + opciones)


In [ ]:
# Métodos para entrenar el reconocedor
# Estos métodos crean la instancia, escriben y leen el set de datos para poder reconocer las imagenes solicitadas
if opciones == "EigenFaces":
  face_recognizer = cv2.face.EigenFaceRecognizer_create()
  face_recognizer.write('modeloEigenFace.xml')
  face_recognizer.read('modeloEigenFace.xml')
  certain = 5700
if opciones == "LBPH":
  face_recognizer = cv2.face.LBPHFaceRecognizer_create()
  face_recognizer.write('modeloLBPHFace.xml')
  face_recognizer.read('modeloLBPHFace.xml')
  certain = 70
else:
  print("La opción seleccionada no es valida")

# Entrenando el reconocedor de rostros
face_recognizer.train(facesData, np.array(labels))

# Barra de carga para una espera más agradable
print("Entrenando...")
for i in tqdm(range(10000)):
  print("", end = '\r')
print("Entrenamiento completado al 100%")
print()
print("Modelo almacenado exitosamente")

## **Probando EigenFaces y LBPH para el reconocimiento facial**

In [ ]:
# Se indica la carpeta donde se almacenaron las imagenes de entrenamiento
dataPath = 'Reconocimiento Facial/Data'
# Mostrar el nombre de la persona solamente
imagePaths = os.listdir(dataPath)
name = imagePaths[0]

# Iniciar la transmisión de vídeo desde la cámara web
video_stream()
# Etiqueta para el vídeo
label_html = 'Capturando...'
# Inicializar el cuadro delimitador para que esté vacío
bbox = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convertir la respuesta JS en imagen OpenCV
    img = js_to_image(js_reply["img"])

    # Crear una superposición transparente para el cuadro delimitador
    bbox_array = np.zeros([480,640,4], dtype = np.uint8)

    # Imagen en escala de grises para la detección de rostros
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Obtener las coordenadas de la región de la cara
    faces = face_cascade.detectMultiScale(gray)
    # Obtener el cuadro delimitador de la cara para la superposición
    for (x,y,w,h) in faces:
      # Obtiene el área donde se ubica el rostro 
      result = face_recognizer.predict(gray)
      # Muestra el resultado en una escala definida para saber que tan probable es que ese sea el rostro de la persona almacenada
      cv2.putText(bbox_array,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
      # Si está dentro del rango de certeza muestra el nombre de la persona que se considera puede ser
      if result[1] < certain:        
          cv2.putText(bbox_array,'{}'.format(name),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
          bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(0,255,0),2)
      # Si no está dentro del rango simplemente lo muestra como persona desconocida
      else:  
          cv2.putText(bbox_array,'Desconocido',(x,y-20),2,0.8,(219,68,55),1,cv2.LINE_AA)
          bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(219,68,55),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # Convertir la superposición de bbox en bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    # Actualizar bbox para que el siguiente cuadro tenga una nueva superposición
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

---
# **Detección de mentiras**

Para este proyecto el objetivo fue detectar las iris de las personas, esto con el fin de detectar mentiras según la posición.

"*Los ojos nos hablan, pero a veces no sabemos interpretar lo que nos están diciendo. Con un entrenamiento adecuado podremos detectar hasta el más sutil de los detalles.*"

# **Detección De Ojos**

Para la detección de ojos simplemente reutilizamos el código anterior del video y de funciones de ayuda para que el recuadro de los ojos se guarde como una imagen los cuales serán capturados con ayuda de otro modelo pre-entrenado de Haar Cascade que nos permitirá reconocerlos y así mostrarlos en tiempo real.

In [ ]:
# JavaScript para crear correctamente nuestro flujo de vídeo en directo utilizando nuestra cámara web como entrada
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Estado:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No existen datos';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: green; font-weight: bold;">' +
          'Cuando haya terminado, haga clic aquí o en el vídeo para detener esta demostración</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox, bbox_eye_left, bbox_eye_right):
  data = eval_js('stream_frame("{}", "{}", "{}", "{}")'.format(label, bbox, bbox_eye_left, bbox_eye_right))
  return data

In [ ]:
# Se indica la carpeta donde se almacenaron las imagenes de entrenamiento
dataPath = 'Reconocimiento Facial/Data'
# Mostrar el nombre de la persona solamente
imagePaths = os.listdir(dataPath)
name = imagePaths[0]

# Iniciar la transmisión de vídeo desde la cámara web
video_stream()
# Etiqueta para el vídeo
label_html = 'Capturando...'
# Inicializar el cuadro delimitador para que esté vacío
bbox = ''
bbox_eye_left = ''
bbox_eye_right = ''
count = 0 
while True:
    js_reply = video_frame(label_html, bbox, bbox_eye_left, bbox_eye_right)
    if not js_reply:
        break

    # Convertir la respuesta JS en imagen OpenCV
    img = js_to_image(js_reply["img"])

    # Crear una superposición transparente para el cuadro delimitador
    bbox_array = np.zeros([480,640,4], dtype = np.uint8)
    bbox_array_eye_left = bbox_array
    bbox_array_eye_right = bbox_array

    # Imagen en escala de grises para la detección de rostros
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Obtener las coordenadas de la región de la cara
    faces = face_cascade.detectMultiScale(gray)
    # Obtener el cuadro delimitador de la cara para la superposición
    for (x,y,w,h) in faces:
      # Obtiene el área donde se ubica el rostro 
      result = face_recognizer.predict(gray)
      # Muestra el resultado en una escala definida para saber que tan probable es que ese sea el rostro de la persona almacenada
      cv2.putText(bbox_array,'{}'.format(result),(x,y-5),1,1.3,(255,255,0),1,cv2.LINE_AA)
      # Si está dentro del rango de certeza muestra el nombre de la persona que se considera puede ser
      if result[1] < certain:        
          cv2.putText(bbox_array,'{}'.format(name),(x,y-25),2,1.1,(0,255,0),1,cv2.LINE_AA)
          bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(0,255,0),2)
      # Si no está dentro del rango simplemente lo muestra como persona desconocida
      else:  
          cv2.putText(bbox_array,'Desconocido',(x,y-20),2,0.8,(219,68,55),1,cv2.LINE_AA)
          bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(219,68,55),2)

    # Obtener las coordenadas de la región del ojo izquierdo
    eye_left = eye_left_cascade.detectMultiScale(gray)
    # Obtener el cuadro delimitador de los ojos para la superposición
    for (x,y,w,h) in eye_left:
      bbox_array_eye_left = cv2.rectangle(bbox_array_eye_left,(x,y),(x+w,y+h),(50, 100, 0),2)

    # Obtener las coordenadas de la región del ojo izquierdo
    eye_right = eye_right_cascade.detectMultiScale(gray)
    # Obtener el cuadro delimitador de los ojos para la superposición
    for (x,y,w,h) in eye_right:
      bbox_array_eye_right = cv2.rectangle(bbox_array_eye_right,(x,y),(x+w,y+h),(50, 100, 0),2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_array_eye_left[:,:,3] = bbox_array[:,:,3]
    bbox_array_eye_right[:,:,3] = bbox_array[:,:,3]

    # Convertir la superposición de bbox en bytes
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox_bytes_eye_left = bbox_eye_left_to_bytes(bbox_array_eye_left)
    bbox_bytes_eye_right = bbox_eye_right_to_bytes(bbox_array_eye_right)

    # Actualizar bbox para que el siguiente cuadro tenga una nueva superposición
    bbox = bbox_bytes
    bbox_eye_left = bbox_bytes_eye_left
    bbox_eye_right = bbox_bytes_eye_right

# **Detección De Mentiras** (En construcción)

El algoritmo trata de sacar un punto promedio, este es el centro de la pupila que se obtiene de un **"entrenamiento"** previo (con el ojo estático), después del entrenamiento se hace una resta del punto centro de la pupila detectada menos el punto promedio y si el resultado es muy grande (sobrepasa un umbral) significa que el ojo esta en otra posición = NOS MIENTE.

## TODO
1. Detectar bien los círculos (pupilas).
2. Dimensionar la imagen original para mayor velocidad de procesamiento.
3. Hacer los radios de los círculos adaptativos dependiendo del área que ocupan los ojos.
4. Detectar dilatación. (Mejora)
5. Agregar marcas de tiempo para establecer los momentos que el sujeto pueda estar mintiendo o no. (Mejora)

In [ ]:
print("%s: Todavía no sé reconocer cuando me mienten" % now)